In [6]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import Optional

# ============================================================================
# 1. Define the output schema using Pydantic
# ============================================================================
class Answer(BaseModel):
    """Schema for sentiment analysis output"""
    sentiment: str = Field(description="sentiment label: positive, negative, or neutral")
    score: float = Field(description="confidence score between 0 and 1")

# ============================================================================
# 2. Create JSON output parser
# JsonOutputParser works better with Ollama than PydanticOutputParser
# ============================================================================
parser = JsonOutputParser(pydantic_object=Answer)

# ============================================================================
# 3. Initialize Ollama LLM
# format="json" tells Ollama to output JSON format
# temperature=0 makes output more deterministic
# ============================================================================
llm = ChatOllama(
    model="llama3.2:latest",
    format="json",  # This is KEY for getting JSON output from Ollama
    temperature=0
)

# ============================================================================
# 4. Create prompt template
# We explicitly ask for JSON format and provide the schema structure
# ============================================================================
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a sentiment analysis expert. 
Analyze the sentiment of the given text and respond ONLY with valid JSON.

{format_instructions}

Always respond with valid JSON matching the schema."""),
    ("user", "Text: {text}")
])

# Inject the format instructions into the prompt
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# ============================================================================
# 5. Build the chain: prompt -> llm -> parser
# The | operator chains these components together
# ============================================================================
chain = prompt | llm | parser

# ============================================================================
# 6. Test the chain with different examples
# ============================================================================
if __name__ == "__main__":
    # Test cases
    test_texts = [
        "I love this product! It's absolutely amazing!",
        "This is the worst experience I've ever had.",
        "The weather is okay today."
    ]
    
    print("=" * 70)
    print("SENTIMENT ANALYSIS RESULTS")
    print("=" * 70)
    
    for text in test_texts:
        try:
            # Invoke the chain with the input text
            result = chain.invoke({"text": text})
            
            print(f"\nText: {text}")
            print(f"Sentiment: {result['sentiment']}")
            print(f"Confidence: {result['score']}")
            print("-" * 70)
            
        except Exception as e:
            print(f"\nError processing: {text}")
            print(f"Error: {str(e)}")
            print("-" * 70)

SENTIMENT ANALYSIS RESULTS

Text: I love this product! It's absolutely amazing!
Sentiment: positive
Confidence: 0.9
----------------------------------------------------------------------

Text: This is the worst experience I've ever had.
Sentiment: negative
Confidence: 0.8
----------------------------------------------------------------------

Text: The weather is okay today.
Sentiment: neutral
Confidence: 0.5
----------------------------------------------------------------------
